# INITIALISING DB

In [ ]:
def init_connection():
    """Create a database connection that persists across Streamlit reruns"""
    try:
        # Runs locally
        engine = create_engine('sqlite:///database.db') 
        # Test the connection
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        logger.info("Database connection successful")
        return engine
    except Exception as e:
        logger.error(f"Error connecting to database: {e}")
        st.error("Failed to connect to database")
        return None

In [ ]:
def initialize_metadata():
    """Initialize database schema if it doesn't exist"""
    engine = init_connection()
    if not engine:
        return None, None, None, None

    metadata = MetaData()

    # Define tables
    residents = Table(
        'residents', metadata,
        Column('id', Integer, primary_key=True),
        Column('first_name', String),
        Column('last_name', String),
        Column('room_num', Integer)
    )

    care_notes = Table(
        'care_notes', metadata,
        Column('id', Integer, primary_key=True),
        Column('resident_id', Integer, ForeignKey('residents.id')),
        Column('note_text', String),
        Column('staff_name', String),
        Column('sentiment_score', Float),
        Column('timestamp', DateTime)
    )

    try:
        metadata.create_all(engine)
        logger.info("Database schema created successfully")
        return engine, metadata, residents, care_notes
    except Exception as e:
        logger.error(f"Error creating database schema: {e}")
        st.error("Failed to create database schema")
        return None, None, None, None


In [ ]:
def generate_care_note():
    """Generate a sample care note with sentiment"""
    positive_notes = [
        "Had a great appetite at breakfast and enjoyed socializing",
        "Participated enthusiastically in morning exercise class",
        "Spent time in the garden and appeared very cheerful",
        "Welcomed family visitors and had an engaging conversation"
    ]

    neutral_notes = [
        "Regular morning routine completed as usual",
        "Attended lunch in dining room",
        "Rested during afternoon quiet time",
        "Watched television in common area"
    ]

    negative_notes = [
        "Seemed less interested in breakfast than usual",
        "Appeared somewhat withdrawn during group activities",
        "Reported feeling tired during the afternoon",
        "Required extra encouragement to participate in activities"
    ]
    
    note_type = random.choices(['positive', 'neutral', 'negative'], weights=[0.5, 0.3, 0.2])[0]
    
    # Skew the fake results so around half are positive, around 30% are neutral and 20% are negative

    if note_type == 'positive':
        note = random.choice(positive_notes)
        sentiment = random.uniform(0.5, 1.0)
    elif note_type == 'negative':
        note = random.choice(negative_notes)
        sentiment = random.uniform(-1.0, -0.2)
    else:
        note = random.choice(neutral_notes)
        sentiment = random.uniform(-0.2, 0.5)
        
    return note, sentiment

def populate_sample_data():
    """Populate the database with sample data if it's empty"""
    engine = init_connection()
    if not engine:
        return

    faker = Faker()
    
    try:
        with engine.connect() as connection:
            # Check if we already have residents
            result = connection.execute(text("SELECT COUNT(*) FROM residents")).scalar()
            
            if result == 0:
                # Start a transaction for sample data
                with connection.begin():
                    # Add sample residents
                    for _ in range(10):
                        connection.execute(
                            text("INSERT INTO residents (first_name, last_name, room_num) VALUES (:first, :last, :room)"),
                            {
                                "first": faker.first_name(),
                                "last": faker.last_name(),
                                "room": random.randint(100, 999)
                            }
                        )
                    
                    # Get all resident IDs
                    resident_ids = [row[0] for row in connection.execute(text("SELECT id FROM residents"))]
                    
                    # Add sample care notes
                    for resident_id in resident_ids:
                        for _ in range(5):
                            note_text, sentiment = generate_care_note()
                            connection.execute(
                                text("""
                                    INSERT INTO care_notes 
                                    (resident_id, note_text, staff_name, sentiment_score, timestamp)
                                    VALUES (:id, :note, :staff, :sentiment, :timestamp)
                                """),
                                {
                                    "id": resident_id,
                                    "note": note_text,
                                    "staff": faker.name(),
                                    "sentiment": sentiment,
                                    "timestamp": datetime.now() - timedelta(days=random.randint(0, 30))
                                }
                            )
                logger.info("Sample data populated successfully")
    except Exception as e:
        logger.error(f"Error populating sample data: {e}")
        st.error("Failed to populate sample data")


In [ ]:
def add_resident(first_name, last_name, room_num):
    """Add a new resident to the database"""
    engine = init_connection()
    if not engine:
        return False

    try:
        with engine.connect() as conn:
            with conn.begin():
                conn.execute(
                    text("INSERT INTO residents (first_name, last_name, room_num) VALUES (:first, :last, :room)"),
                    {"first": first_name, "last": last_name, "room": room_num}
                )
        logger.info(f"Added resident: {first_name} {last_name}")
        return True
    except Exception as e:
        logger.error(f"Error adding resident: {e}")
        return False


In [ ]:
def add_care_note(resident_id, note_text, staff_name):
    """
    Add a new care note to the database with automatically calculated sentiment score
    """
    engine = init_connection()
    if not engine:
        return False

    try:
        # Calculate sentiment score automatically
        sentiment_score = analyse_sentiment(note_text)
        
        with engine.connect() as conn:
            with conn.begin():
                conn.execute(
                    text("""
                        INSERT INTO care_notes 
                        (resident_id, note_text, staff_name, sentiment_score, timestamp)
                        VALUES (:id, :note, :staff, :sentiment, :timestamp)
                    """),
                    {
                        "id": resident_id,
                        "note": note_text,
                        "staff": staff_name,
                        "sentiment": sentiment_score,
                        "timestamp": datetime.now()
                    }
                )
        logger.info(f"Added care note for resident {resident_id} with sentiment score {sentiment_score}")
        return True, sentiment_score
    except Exception as e:
        logger.error(f"Error adding care note: {e}")
        return False, None


In [ ]:
def get_care_notes(resident_id):
    """Fetch care notes for a specific resident with formatted sentiment descriptions"""
    engine = init_connection()
    if not engine:
        return pd.DataFrame()

    try:
        with engine.connect() as conn:
            df = pd.read_sql(
                """
                SELECT 
                    id,
                    resident_id,
                    note_text,
                    staff_name,
                    sentiment_score,
                    timestamp
                FROM care_notes 
                WHERE resident_id = :id 
                ORDER BY timestamp DESC
                """,
                conn,
                params={"id": resident_id}
            )
            
            # Add a human-readable sentiment description
            def get_sentiment_description(score):
                if score >= 0.5:
                    return "Very Positive"
                elif score >= 0.1:
                    return "Positive"
                elif score <= -0.5:
                    return "Very Negative"
                elif score <= -0.1:
                    return "Negative"
                else:
                    return "Neutral"
            
            df['sentiment_description'] = df['sentiment_score'].apply(get_sentiment_description)
            return df
            
    except Exception as e:
        logger.error(f"Error fetching care notes: {e}")
        return pd.DataFrame()
